# Plan

- 







In [1]:
!pip install bpe

     |████████████████████████████████| 21.0MB 1.3MB/s 
     |████████████████████████████████| 358kB 57.1MB/s 
     |████████████████████████████████| 747kB 41.7MB/s 


In [2]:
!pip install tqdm 

In [3]:
# imports


from google.colab import files
import pandas as pd
import io
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np
import string
from keras.utils import to_categorical
from bpe import Encoder
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import notebook

# from random import shuffle
import random

In [4]:
# load data
uploaded = files.upload()

Saving babynames-clean.csv to babynames-clean.csv


In [5]:
names = pd.read_csv(io.BytesIO(uploaded['babynames-clean.csv']))
names.drop(names.columns[[1, 1]], axis=1, inplace=True) # drop genders

In [6]:
##########################
#         BPE            #
##########################

corpus = names["John"].tolist() # to list ify

encoder = Encoder(800, pct_bpe=1)  # encode it all
encoder.fit(corpus) # fit it to the corpus

names["tokenized_names"] = names.apply(lambda row:encoder.tokenize(row["John"]), axis=1) # make the collumn

names.head()


,John,tokenized_names
0,William,"[__sow, wi, ll, ia, m, __eow]"
1,James,"[__sow, ja, me, s, __eow]"
2,Charles,"[__sow, ch, ar, le, s, __eow]"
3,George,"[__sow, ge, or, ge, __eow]"
4,Frank,"[__sow, fr, an, k, __eow]"


In [7]:
## find max_len

# high = 0 # initialize high
# high = names.apply(lambda row: max(high, len(row["tokenized_names"])), axis=1) # jank, could cut the max func here
# max_len = max(high.tolist()) # take the highest len value
# print(max_len)


In [8]:
##########################
#       Padding          #
##########################

# names["padded"] = names.apply(lambda row: row["tokenized_names"]+(max_len - len(row["tokenized_names"]))*["__pad"], axis=1) # pad em

# names.head()


In [9]:
##########################
#       Tokenize         #
##########################

tok = defaultdict(lambda: len(tok)) # define the default dict. add one to deal with new keys

names["toked"] = [[tok[j] for j in i] for i in names["tokenized_names"].tolist()] # tokenize it!

names.head()


,John,tokenized_names,toked
0,William,"[__sow, wi, ll, ia, m, __eow]","[0, 1, 2, 3, 4, 5]"
1,James,"[__sow, ja, me, s, __eow]","[0, 6, 7, 8, 5]"
2,Charles,"[__sow, ch, ar, le, s, __eow]","[0, 9, 10, 11, 8, 5]"
3,George,"[__sow, ge, or, ge, __eow]","[0, 12, 13, 12, 5]"
4,Frank,"[__sow, fr, an, k, __eow]","[0, 14, 15, 16, 5]"


In [10]:
##########################
#       One Hot          #
##########################

names["one_hot"] = names.apply(lambda row: to_categorical(row["toked"], num_classes=len(tok)), axis=1) # one hot it! num_classes is len of tok 

names.head()

,John,tokenized_names,toked,one_hot
0,William,"[__sow, wi, ll, ia, m, __eow]","[0, 1, 2, 3, 4, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,James,"[__sow, ja, me, s, __eow]","[0, 6, 7, 8, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Charles,"[__sow, ch, ar, le, s, __eow]","[0, 9, 10, 11, 8, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,George,"[__sow, ge, or, ge, __eow]","[0, 12, 13, 12, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Frank,"[__sow, fr, an, k, __eow]","[0, 14, 15, 16, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [11]:
# # data = names["one_hot"].tolist()

# # print(data[:3])

# arr = [[[1,2,3], [4,5,6]], [[7,8,9], [10, 11, 12]], [[13, 14, 15]]]
# # print(a)
# a = []
# b = []
# fliper = True


# for i in arr:
#   for j in i:
#     # if flip:
#     #   a.append(j)
#     # else:
#     #   b.append(j)
#     # flip = !flip

#     if fliper == True:
#       a.append(j)
#       fliper = False

#     else:
#       b.append(j)
#       fliper = True


# print(a)
# print(b)

In [12]:
########################
#     Markhov Time     #
########################

data = names["one_hot"].tolist()

inputs = []
labels = []

fliper = True 
for i in data:
  for j in i:
    if fliper == True:
        inputs.append(list(j))
        fliper = False
    else:
      labels.append(list(j))
      fliper = True

max_len = min(len(inputs), len(labels))

inputs = inputs[:max_len]
labels = labels[:max_len]

joined_lists = list(zip(inputs, labels))
random.shuffle(joined_lists) # Shuffle "joined_lists" in place
inputs, labels = zip(*joined_lists) # Undo joining


a = np.array(inputs)
print(np.shape(a))

b = np.array(labels)
print(np.shape(b))

(17672, 433)
(17672, 433)


In [13]:
#################
#     Batches   #
#################
inp_batches = []
lab_batches = []
inp = []
lab = []


for i in range(min(len(inputs), len(labels))):
  if len(inp) == 64:
    inp_batches.append(inp)
    lab_batches.append(lab)

    inp = []
    lab = []
  inp.append(inputs[i])
  lab.append(labels[i])
  #   batches.append(b)
  #   b = []
  # b.append(list(i))
  # if len()
  # pass

a = np.array(inp_batches)
print(np.shape(a))

b = np.array(lab_batches)
print(np.shape(b)) 


inp_ts = torch.FloatTensor(inp_batches)
lab_ts = torch.FloatTensor(lab_batches)

# print(ts.shape)

print(inp_ts.shape)
print(lab_ts.shape)
# print(ts)



(276, 64, 433)
(276, 64, 433)
torch.Size([276, 64, 433])
torch.Size([276, 64, 433])


In [14]:
#####################
#   df to tensor    #
#####################

# data = torch.from_numpy(np.array(names["one_hot"].tolist()))

# print(data)


-





In [15]:
###################################
#     Model :clap: Time :clap:    #
###################################


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # layers
        self.layer1 = nn.Linear(433, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 256)
        self.layer4 = nn.Linear(256, 434)
        self.output_ac = nn.Softmax(dim=1) # uh 

    def forward(self, x):
        # model structure
        net = self.layer1(x)
        net = self.layer2(net)
        net = self.layer3(net)
        net = self.layer4(net)
        net = self.output_ac(net)

        return net


net = Net().cuda()




In [18]:
###################
#   OPTIMIZER     #
###################


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

net.apply(init_weights)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if sys.path[0] == '':


Net(
  (layer1): Linear(in_features=433, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=256, bias=True)
  (layer4): Linear(in_features=256, out_features=434, bias=True)
  (output_ac): Softmax(dim=1)
)

In [19]:
###################
#     TRAINING    #
###################

zach = notebook.tqdm(range(100))

for epoch in zach:  # loop over the dataset multiple times
  for batch in range(len(inp_ts)):
    # for i in range(len(inp_batches[batch])):

    inputs = torch.FloatTensor(inp_batches[batch]).cuda()
    labels = torch.FloatTensor(lab_batches[batch]).cuda()

    # print(torch.argmax(labels, dim=1))
    # print(inp_batches[batch])
    # print(inputs.size())
    # print(inp_batches[30])

    # print(inputs)

    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, torch.argmax(labels, dim=1))
    loss.backward()
    optimizer.step()

    #       # print statistics
    # # running_loss += loss.item()
    # # if batch % 2000 == 1999:    # print every 2000 mini-batches
    # if True:
    #   print('[%d, %5d] loss: %.3f' %
    #               (epoch + 1, batch + 1, loss.item()))
    #   running_loss = 0.0
  zach.set_description(f'loss:{loss.item()}')

  # print("finished training")

In [29]:
print(tok)

defaultdict(<function <lambda> at 0x7f20d88b4378>, {'__sow': 0, 'wi': 1, 'll': 2, 'ia': 3, 'm': 4, '__eow': 5, 'ja': 6, 'me': 7, 's': 8, 'ch': 9, 'ar': 10, 'le': 11, 'ge': 12, 'or': 13, 'fr': 14, 'an': 15, 'k': 16, 'jo': 17, 'se': 18, 'ph': 19, 'th': 20, 'om': 21, 'as': 22, 'he': 23, 'nr': 24, 'y': 25, 'ro': 26, 'be': 27, 'rt': 28, 'ed': 29, 'wa': 30, 'rd': 31, 'ha': 32, 'rr': 33, 'lt': 34, 'er': 35, 'ur': 36, 'al': 37, 'sa': 38, 'mu': 39, 'el': 40, 'da': 41, 'vi': 42, 'd': 43, 'lo': 44, 'ui': 45, 'e': 46, 'li': 47, 'cl': 48, 'en': 49, 'ce': 50, 'ri': 51, 'dr': 52, 'ew': 53, 'ni': 54, 'ne': 55, 'st': 56, 'je': 57, 'ss': 58, 'os': 59, 'ca': 60, 'r': 61, 'pe': 62, 'te': 63, 'nj': 64, 'am': 65, 'in': 66, 'ic': 67, 'ie': 68, 'rb': 69, 't': 70, 'co': 71, 'b': 72, 'to': 73, 'rl': 74, 'ho': 75, 'ma': 76, 'mi': 77, 'ae': 78, 'l': 79, 'rm': 80, 'ji': 81, 'ci': 82, 'rv': 83, 'ey': 84, 'ea': 85, 'eu': 86, 'ra': 87, 'lp': 88, 'h': 89, 'au': 90, 'de': 91, 'n': 92, 'pa': 93, 'ul': 94, 'ga': 95, 'is'

In [32]:
def get_key(val): 
    for key, value in tok.items(): 
         if val == value: 
             return key 
  
    return "key doesn't exist"


inpt = torch.FloatTensor(inp_batches[6]).cuda()
oupt = net(inpt)

def decode(oup):
  n = []
  amx = torch.argmax(oup, dim = 1)
  # print(amx)
  for i in amx:
    # print(tok[int(i)])
    n.append(get_key(int(i)))
  print(n)

decode(oupt)



['__eow', '__eow', '__eow', '__eow', 'n', '__eow', 'ma', '__eow', '__eow', 'n', 'n', '__eow', '__eow', '__eow', 'ma', 'ma', '__eow', 'ma', '__eow', '__eow', 'ma', 'ma', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', 'ma', '__eow', 'ma', 'ma', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', 'n', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', '__eow', 'ma', '__eow', 'ma', '__eow', 'ma', 'ma']


In [20]:
###################
#     TRAINING    #
###################

# zach = notebook(range(100))

# for epoch in zach:  # loop over the dataset multiple times
#   for batch in range(len(inp_ts)):
#       running_loss = 0.0
#       # for i, data in enumerate(batch):
#       # for i in range(len(batch)):

#           # get the inputs; data is a list of [inputs, labels]
#           # inputs, labels = data
#           # inputs = data[0]
#           # labels = data[1]
#           # print(ts)
#           print(batch.shape)
#           # print("labels", labels)
#           # print("inputs", inputs)
#           # print(torch.argmax(labels, dim=0))

#           # zero the parameter gradients
#           optimizer.zero_grad()

#           # forward + backward + optimize
#           outputs = net(inputs)
#           # print(outputs)
#           loss = criterion(outputs, labels)
#           loss.backward()
#           optimizer.step()

#           # print statistics
#           # running_loss += loss.item()
#           # if i % 2000 == 1999:    # print every 2000 mini-batches
#           #     print('[%d, %5d] loss: %.3f' %
#           #           (epoch + 1, i + 1, running_loss / 2000))
#           #     running_loss = 0.0
#   zach.set_description(f'loss:{loss.item()}')

#   print('Finished Training')




IndentationError: ignored